In [57]:
import pandas as pd
import numpy as np
import yfinance as yf

#### Getting the data

In [58]:
symbol = input("Enter the symbol you want to analyze: ")

In [59]:
ticker = yf.Ticker(symbol)

In [60]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

#### Calculating P/E ratio
    - using diluted EPS

In [61]:
latest_price = float(ticker.history(period='1d')['Close'].values[-1])

In [62]:
ratios = latest_price / Income_statement['Diluted EPS']

In [63]:
ratios = pd.DataFrame(ratios)
ratios.rename({'Diluted EPS': 'P/E ratio'}, axis =1, inplace=True )

#### Calculating price/Book ratio

In [64]:
market_capitalization = Balance_sheet['Ordinary Shares Number'] * latest_price
# calculating net book value 
current_liabilities = Balance_sheet['Current Liabilities'] + Balance_sheet['Other Current Liabilities'] + Balance_sheet['Current Deferred Liabilities']
non_current_liabilities = Balance_sheet['Non Current Deferred Liabilities'] + Balance_sheet['Other Non Current Liabilities'] + Balance_sheet['Non Current Deferred Liabilities']
Balance_sheet['Total Liabilities'] = current_liabilities + non_current_liabilities
Net_book_value = Balance_sheet['Total Assets'] - Balance_sheet['Total Liabilities'] - Balance_sheet['Total Equity Gross Minority Interest']


In [65]:
ratios['Price/Book'] = market_capitalization / Net_book_value

In [66]:
ratios

,P/E ratio,Price/Book
2023-06-30,45.15599,287.817969
2022-06-30,45.296372,142.564515
2021-06-30,54.299377,106.404752
2020-06-30,75.88715,70.009725


#### Calculating Price/Sales ratio

In [67]:
ratios['Price/Sales'] = market_capitalization / Income_statement['Total Revenue']
ratios

,P/E ratio,Price/Book,Price/Sales
2023-06-30,45.15599,287.817969,15.329738
2022-06-30,45.296372,142.564515,16.455283
2021-06-30,54.299377,106.404752,19.553032
2020-06-30,75.88715,70.009725,23.139948


#### calculating Dividend Yield

In [68]:
ticker =yf.Ticker(symbol)
Dividends = ticker.dividends

In [69]:
Dividends = pd.DataFrame(pd.to_datetime(Dividends.index), Dividends.values ).reset_index()
Dividends.set_index('Date', inplace=True)
Dividends.rename({'index': 'Quarterly dividends'}, axis = 1, inplace=True)
Dividends = Dividends.groupby(Dividends.index.year)['Quarterly dividends'].sum()
Dividends = pd.DataFrame(Dividends)
Dividends.rename({'Quarterly dividends' : 'Yearly dividends'}, axis=1, inplace=True)
Dividends.sort_index(ascending=False, inplace=True)

In [70]:
Dividends['Dividend yields(%)'] = (Dividends['Yearly dividends'] / latest_price) * 100
Dividends.drop('Yearly dividends', axis =1, inplace=True)
Dividends.head()

,Dividend yields(%)
Date,
2024,0.343163
2023,0.638283
2022,0.581089
2021,0.526183
2020,0.478141


In [71]:
ratios = ratios.merge(Dividends, left_on=ratios.index.year, right_on=Dividends.index)
ratios.rename({'key_0' : 'Year'}, axis=1, inplace=True)
ratios.set_index('Year', inplace=True)
ratios

,P/E ratio,Price/Book,Price/Sales,Dividend yields(%)
Year,,,,
2023,45.15599,287.817969,15.329738,0.638283
2022,45.296372,142.564515,16.455283,0.581089
2021,54.299377,106.404752,19.553032,0.526183
2020,75.88715,70.009725,23.139948,0.478141


In [80]:
company_name = yf.Ticker(symbol).info['longBusinessSummary'].split()[0]

##### Save the multiples by company name

In [81]:
ratios.to_csv(f'{company_name} multiples.csv')